In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
MAX_LEN_SRC = 128
MAX_LEN_TAR = 0

In [ ]:
import operator
import itertools
from pprint import pprint

class DataSequence():

    def __init__(self, dataset, filename):
        self.dataset = dataset
        self.filename = filename
    def getUserSequenes(self):
        train_sequences = dict()
        users = []
        Items = set()
        history = dict()
        user = 0
        nr_movies = 0
        max_len = 0

        watched = {}
        with open(self.filename, "r") as f:
            for line in f:
                user, item, rating, timestamp = line.strip().split("::")
                user = int(user)
                if ( int(float(rating)) >= 3):
                  if int(item) > nr_movies:
                    nr_movies = int(item)
                  if user not in watched:
                    watched[user] = []
                  
                  watched[user].append((int(item), timestamp))

        for key in watched:
          watched[key].sort(key = lambda x: x[1]) 
          if len(watched[key]) > max_len:
            max_len = len(watched[key])
        print("Max reviews by an user " + str(max_len))

        data_X = []
        data_Y = []
        for key in watched:
          if len(watched[key]) > MAX_LEN_SRC+MAX_LEN_TAR:
            latest_data = [x for (x,y) in watched[key]][-(MAX_LEN_SRC+MAX_LEN_TAR+1):]
          else:
            latest_data = [x for (x,y) in watched[key]]


          append_X = latest_data[:-1]
          append_Y = latest_data[1:]

          append_X += [0] * (MAX_LEN_SRC+MAX_LEN_TAR - len(append_X)) 
          append_Y += [0] * (MAX_LEN_SRC+MAX_LEN_TAR - len(append_Y))

          data_X.append(append_X)
          data_Y.append(append_Y)

          
        

        return data_X, data_Y, nr_movies

data_sequence = DataSequence("MovieLens", "/content/drive/MyDrive/sadc/train.dat")
data_X, data_Y, nr_movies = data_sequence.getUserSequenes()



import numpy as np
from sklearn.model_selection import train_test_split

trainX, testX, trainY, testY = train_test_split(data_X, data_Y, test_size=0.1, random_state=42)
 

Max reviews by an user 1564


In [ ]:
len(trainX)

5435

In [ ]:
for x in data_Y:
  if(len(x) != MAX_LEN_SRC):
    print("error")

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import string
import re
from unicodedata import normalize
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential,load_model
from keras.layers import LSTM,Dense,Embedding,RepeatVector,TimeDistributed,Activation,Dropout
from keras.callbacks import EarlyStopping
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from nltk.translate.bleu_score import corpus_bleu
from tensorflow.keras import activations
import pandas as pd
import tensorflow as tf
from string import punctuation
import matplotlib.pyplot as plt
from IPython.display import Markdown, display
def printmd(string):
    # Print with Markdowns    
    display(Markdown(string))

In [ ]:
nr_movies

3952

In [ ]:
src_vocab_size = nr_movies + 1
tar_vocab_size = nr_movies + 1
src_length = MAX_LEN_SRC 
tar_length = MAX_LEN_SRC

print(f'\nTarget Vocabulary Size: {tar_vocab_size}')
print(f'Target Max Length: {tar_length}')

print(f'\nSource Vocabulary Size: {src_vocab_size}')
print(f'Source Max Length: {src_length}\n')


Target Vocabulary Size: 3953
Target Max Length: 128

Source Vocabulary Size: 3953
Source Max Length: 128



In [ ]:
def encode_output(sequences, vocab_size):
    # one hot encode target sequence
    ylist = list()
    for sequence in sequences:
        encoded = to_categorical(sequence, num_classes=vocab_size)
        ylist.append(encoded)
    y = np.array(ylist)
    y = y.reshape(sequences.shape[0], sequences.shape[1], vocab_size)
    return y

In [ ]:
from collections import Counter

def evaluate(test_model):
  total_right10 = 0
  total_right5 = 0
  total_right3 = 0
  total_right1 = 0
  total = 0 
  correct_movies = []
  total_sps10 = 0
  modified_sps = 0

  for idx, x in enumerate(testX[:100]):
    half1, half2 = getHalves(x)
    count = np.count_nonzero(half1)
    user_predictions = []

    source = np.array(half1)
    source = source.reshape((1, source.shape[0]))
    prediction = test_model.predict(source, verbose=0)[0]
    integers2 = [np.argmax(vector) for vector in prediction]
    user_predictions10 = np.argpartition(prediction[count-1], -10)[-10:]
    user_predictions5 = np.argpartition(prediction[count-1], -5)[-5:]
    user_predictions3 = np.argpartition(prediction[count-1], -3)[-3:]
    user_predictions1 = np.argpartition(prediction[count-1], -1)[-1:]
    
    total += 1
    if half2[0] in user_predictions10:
      total_sps10 += 1
    for x in half2[:3]:
      if x in user_predictions10:
        modified_sps += 1
        break;
    
    for pred in user_predictions10:
      if pred in half2:
        correct_movies.append(pred)
        total_right10 += 1
        break
    # for pred in user_predictions5:
    #   if pred in half2:
    #     total_right5 += 1
    #     break
    # for pred in user_predictions3:
    #   if pred in half2:
    #     total_right3 += 1
    #     break
    # for pred in user_predictions1:
    #   if pred in half2:
    #     total_right1 += 1
    #     break
    
  print("Item Coverage: ", len(Counter(correct_movies).keys()))
  print("User Coverage: ", total_right10/total)
  print("SPS1@10: ", total_sps10/total)
  print("SPS3@10: ", modified_sps/total)
  
  return total_sps10/total, len(Counter(correct_movies).keys())
  # print("", total_right5/total)
  # print("", total_right3/total)
  # print("", total_right1/total)

In [ ]:
import tensorflow
print(tensorflow.__version__)

2.7.0


In [ ]:
def getHalves(x):
  length = np.count_nonzero(x)
  half1 = x[:length//2]
  half2 = x[:length][-length//2:]

  # print(len(half1))
  # print(len(half2))

  half1 += [0] * (MAX_LEN_SRC+MAX_LEN_TAR - len(half1))
  # print(len(half1))
  return half1, half2

In [ ]:
from collections import Counter
total = 0
for x in trainX:
  keys = Counter(x).keys() # equals to list(set(words))
  values = Counter(x).values() # counts the elements' frequency
  for key, value in zip(keys, values):
    if value > 1 and key != 0:
      print(value, key)
      break

print(total)

0


In [ ]:
import warnings
warnings.filterwarnings("ignore")

def create_model(src_vocab, tar_vocab, src_timesteps, tar_timesteps, n_units):
    # Create the model
    model = Sequential()
    model.add(Embedding(src_vocab, n_units, input_length=src_timesteps, mask_zero=True))
    model.add(LSTM(120, return_sequences=True))
    model.add(Dropout(0.2))
    model.add(Dense(tar_vocab, activation='softmax'))
    return model
 
# Create model
model = create_model(src_vocab_size, tar_vocab_size, src_length, tar_length, 64)
# opt = tf.optimizers.Adagrad(lr=1, epsilon=1e-3)
opt = tf.optimizers.Adam(lr=0.005)
model.compile(optimizer=opt, loss="categorical_crossentropy")

from random import shuffle
import time

def get_batch(x, y, batch_size):
  c = list(zip(x, y))

  shuffle(c)

  x, y = zip(*c)

  return np.array(x[:batch_size]), encode_output(np.array(y[:batch_size]), tar_vocab_size)

test_X, test_Y = get_batch(testX, testY, 128)


val_loss =[]
train_loss = []
min_val_loss = 1000
max_sps = 0
max_item = 0
saved_model = None

start_time = time.time()


for epoch in range(0,100):
  epoch_start_time = time.time()

  train_res = 0
  batch_size = 32
  batches = int(len(trainX) /batch_size)
  for batch in range(0,batches):
    train_X, train_Y = get_batch(trainX, trainY, batch_size)
    train_res += model.train_on_batch(
      train_X, train_Y
    )

  test_res = model.test_on_batch(
    test_X, test_Y
  )

  sps, item = evaluate(model)
  if sps > max_sps:
    max_sps = sps
    model.save("/content/drive/MyDrive/sadc/testmodels3/max_sps")
  
  if item > max_item:
    max_item = item
    model.save("/content/drive/MyDrive/sadc/testmodels3/max_item")

  val_loss.append(test_res)
  train_loss.append(train_res/batches)
  print("Epoch", epoch,  int(time.time() - epoch_start_time), "seconds","train:", train_res/batches, "validation", test_res , "Total", int(time.time() - start_time), "seconds")



In [ ]:
items_model = tf.keras.models.load_model('/content/drive/MyDrive/sadc/testmodels2/max_item')
sps_model = tf.keras.models.load_model('/content/drive/MyDrive/sadc/testmodels2/max_sps')

In [ ]:


sps_model.save_weights('/content/drive/MyDrive/sadc_model/checkpoint')


In [ ]:
evaluate(items_model)

evaluate(sps_model)